In [ ]:
import multiprocessing
import threading
import tensorflow as tf
from ACNet import Access
from Framework import Framework

In [ ]:
NUMS_CPU = multiprocessing.cpu_count()
state_size = 58
batch_size = 50
action_size = 3
max_episodes = 10
GD = {}

In [ ]:
class Worker(Framework):

    def __init__(self, name, access, state_size, batch_size, action_size):
        super().__init__(name, access, state_size, batch_size, action_size)

    def run(self, sess, max_episodes, t_max=8):
        episode_score_list = []
        episode = 0
        while episode < max_episodes:
            episode += 1
            episode_socre, _ = self.run_episode(sess, t_max)
            episode_score_list.append(episode_socre)
            GD[str(self.name)] = episode_score_list
            if self.name == 'W0':
                print('Episode: %f, score: %f' % (episode, episode_socre))
                print('\n')

In [ ]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# with tf.Session(config=config) as sess:
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        A = Access(state_size, batch_size, action_size)
        F_list = []
        for i in range(NUMS_CPU):
            F_list.append(Worker('W%i' % i, A, state_size, batch_size, action_size))
        COORD = tf.train.Coordinator()
        sess.run(tf.global_variables_initializer())
        sess.graph.finalize()

        threads_list = []
        for ac in F_list:
            job = lambda: ac.run(sess, max_episodes)
            t = threading.Thread(target=job)
            t.start()
            threads_list.append(t)
        COORD.join(threads_list)
        A.save(sess, 'model/saver_1.ckpt')

test

In [ ]:
tf.reset_default_graph()

In [ ]:
import tensorflow as tf
from ACNet import Access, Agent
from env03.main import Account

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
state_size = 58
batch_size = 50
action_size = 3


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    with tf.device("/cpu:0"):
        A = Access(state_size, batch_size, action_size)
        W = Agent('W0', A, state_size, batch_size, action_size)
        A.restore(sess,'model/saver_1.ckpt')
        W.init_network(sess)
        env = Account()
        state = env.reset()
        for _ in range(200):
            state = np.transpose(state, [1, 0, 2])
            action = W.get_deterministic_policy_action(sess, state)
            state, reward, done = env.step(action)

In [ ]:
value, reward = env.plot_data()

pd.Series(value).plot(figsize=(16,6))

In [ ]:
pd.Series(reward).plot(figsize=(16,6))
pd.Series(np.zeros_like(reward)).plot(figsize=(16,6), color='r')